In [6]:
import pandas as pd
import numpy as np 
import torch 
import torch.nn as nn 
from sklearn.feature_extraction.text import TfidfVectorizer


In [3]:
df = pd.read_csv(r"C:\Users\hrida\MyPythonStuff\new_reddit_voc_data.csv", encoding='latin1')

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
df['sentiment'] = np.random.randint(1,11, df.shape[0])
df['body'] = df['body'].fillna('')
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['body'])
df['embeddings'] = list(tfidf_matrix.toarray())

In [11]:
data = df[['embeddings', 'sentiment']]

In [18]:
def convertTorch(array):
    return torch.from_numpy(array)

In [25]:
data['embeddings'] = data['embeddings'].apply(convertTorch)

C:\Users\hrida\AppData\Local\Temp/ipykernel_13416/1249021970.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['embeddings'] = data['embeddings'].apply(convertTorch)


In [21]:
y = torch.tensor(df['sentiment'].values)

In [62]:
class Net(nn.Module):
    def __init__(self, INPUT_SIZE):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(INPUT_SIZE, 32)
        self.fc2 = nn.Linear(32, 8)
        self.fc3 = nn.Linear(8, 3)
        self.fc4 = nn.Linear(3, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        return x

In [63]:
INPUT_SIZE = len(data['embeddings'][0])
model = Net(INPUT_SIZE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
lossF = nn.MSELoss()

In [75]:
embeddings = data['embeddings'].values
embeddings_tensor = torch.stack([torch.tensor(embedding) for embedding in embeddings])
sentiment_labels = data['sentiment'].values
# embeddings_tensor = torch.stack(embeddings)
sentiment_labels_tensor = torch.tensor(sentiment_labels, dtype =float)  # Assuming sentiment labels are integers
from torch.utils.data import TensorDataset

dataset = TensorDataset(embeddings_tensor, sentiment_labels_tensor)

C:\Users\hrida\AppData\Local\Temp/ipykernel_13416/2412137876.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_tensor = torch.stack([torch.tensor(embedding) for embedding in embeddings])


In [83]:
embeddings_tensor.dtype,sentiment_labels_tensor.dtype

(torch.float64, torch.float64)

In [85]:
from torch.utils.data import DataLoader
batch_size = 2  

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
for epoch in range(100):
    for batch in data_loader:
        embeddings_batch, sentiment_labels_batch = batch
        
        embeddings_batch = embeddings_batch.to(model.fc1.weight.dtype)
        sentiment_labels_batch = sentiment_labels_batch.float()
        outputs = model(embeddings_batch)
        outputs = outputs.float()
        loss = lossF(outputs, sentiment_labels_batch)
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

c:\Users\hrida\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\hrida\anaconda3\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(3.3027, grad_fn=<MseLossBackward0>)
tensor(1.7064, grad_fn=<MseLossBackward0>)
tensor(8.0575, grad_fn=<MseLossBackward0>)
tensor(3.9672, grad_fn=<MseLossBackward0>)
tensor(8.5336, grad_fn=<MseLossBackward0>)
tensor(1.2632, grad_fn=<MseLossBackward0>)
tensor(2.6794, grad_fn=<MseLossBackward0>)
tensor(5.5078, grad_fn=<MseLossBackward0>)
tensor(1.5598, grad_fn=<MseLossBackward0>)
tensor(0.1583, grad_fn=<MseLossBackward0>)
tensor(6.7016, grad_fn=<MseLossBackward0>)
tensor(0.2291, grad_fn=<MseLossBackward0>)
tensor(10.0853, grad_fn=<MseLossBackward0>)
tensor(1.5934, grad_fn=<MseLossBackward0>)
tensor(0.6715, grad_fn=<MseLossBackward0>)
tensor(0.4625, grad_fn=<MseLossBackward0>)
tensor(7.7601, grad_fn=<MseLossBackward0>)
tensor(7.8073, grad_fn=<MseLossBackward0>)
tensor(3.2281, grad_fn=<MseLossBackward0>)
tensor(8.4379, grad_fn=<MseLossBackward0>)
tensor(0.9872, grad_fn=<MseLossBackward0>)
tensor(3.1525, grad_fn=<MseLossBackward0>)
tensor(3.3317, grad_fn=<MseLossBackward0>)
tensor(0.7

In [91]:
def embeddings(embeddings_tensor):
    """Generates embeddings. Have kept them 3-d for visuzalization purposes

    Args:
        embeddings_tensor (_type_): embeddings_tensor as made above

    Returns:
        torch.tensor: 3-d
    """

    model.eval()
    with torch.no_grad():
        input_data = embeddings_tensor[0].double()
        
        model.fc1.weight = torch.nn.Parameter(model.fc1.weight.double())
        model.fc1.bias = torch.nn.Parameter(model.fc1.bias.double())
        model.fc2.weight = torch.nn.Parameter(model.fc2.weight.double())
        model.fc2.bias = torch.nn.Parameter(model.fc2.bias.double())
        model.fc3.weight = torch.nn.Parameter(model.fc3.weight.double())
        model.fc3.bias = torch.nn.Parameter(model.fc3.bias.double())
        output_fc3 = model.fc3(torch.relu(model.fc2(torch.relu(model.fc1(input_data)))))
        
    return output_fc3


In [92]:
output_fc3

tensor([ 3.0607,  3.7715, -0.3897], dtype=torch.float64)